In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt
import math 
import random
%matplotlib inline

In [2]:
def GetMasses(nfields,g_beta,g_sigma,m_mu,m_sigma):
    excess = 2
    nfields = nfields + excess
    M = np.zeros((nfields,nfields))
    G = np.zeros((nfields,nfields))
    K = np.zeros((nfields,math.floor(nfields/g_beta)))
    for i in range(0, nfields):
        for j in range(0, nfields):
            M[i,j]=random.gauss(m_mu,m_sigma)
        for m in range(0,nfields):
            M[m,i] = M[i,m]
        for z in range(0,math.floor(nfields/g_beta)):
            K[i,z] = random.gauss(0,g_sigma)/nfields
    G = np.dot(K,np.transpose(K))
    wG, vG = np.linalg.eigh(G)
    
    Mp = np.dot(np.transpose(vG),M)
    Mp = np.dot(Mp,vG)
    
    wMp,vMp = np.linalg.eig(Mp)
    
    Md = np.dot(np.transpose(vMp),Mp)
    Md = np.dot(Md,vMp)
    
    for i in range(0, nfields):
        for j in range(0, nfields):
            Md[i,j]=Md[i,j] / (np.sqrt(wG[i])*np.sqrt(wG[j]))
    wMd,vMd = np.linalg.eig(Md)
    for i in range(0,nfields):
        wMd[i] = -abs(wMd[i])*10**(-20)
    #print(wMd)
    wMd = wMd[excess:]
    #print(wMd)
    return(wMd)

In [3]:
def GetPhi0s(nfields,efolds_goal,vevs,s,r):
    #print("started the phi_sampling")
    
    expscale = np.exp((4*efolds_goal/nfields - 1/9*(s+r)**3*(3*np.log(s+r)-1)/r + 1/9*s**3*(3*np.log(s)-1)/r) / ((s+r)**3/3/r-s**3/3/r) - 0.5772156649015328606065120)
    y = np.zeros(nfields)
    randint = math.floor(random.random()*nfields)
    boolean = 0
    while boolean == 0:
        vevs = np.random.uniform(s,s+r,nfields)
        for i in range(0,nfields):
            if i != randint:
                y[i] = -np.log(random.random())/expscale
        tempsum = 0 
        for i in range(0,nfields):
            if i != randint:
                tempsum = tempsum + vevs[i]**2*np.log(vevs[i]/y[i])
        #print(tempsum)
        if 0.25*tempsum < efolds_goal and 0.25*tempsum/efolds_goal > 0.98:
            y[randint] = vevs[randint] / (np.exp(4*efolds_goal/(vevs[randint]**2) - tempsum / (vevs[randint]**2)))
            boolean = 1
            #print("Success!")
            #print(tempsum)
            #print("scale =",expscale)
    #print("ended the phi_sampling")
    return(y,vevs)
        
        
        
        

In [4]:
def dNdphi_SR(phi_pivot,phi_end,mass2,vev):
    eps = eps_SR(phi_pivot,mass2,vev)
    V_i = V_i_sum_sep(phi_pivot,mass2,vev,nfields)
    Z_i = Z_i_BE(phi_end,mass2,vev,nfields)
    V = potential(phi,mass2,vev)
    dNdphi = ((1/np.sqrt(2*eps))/V)*(V_i +Z_i)
    #print("eps,V,V_i,Z-i",eps,V,V_i,Z_i)
    return dNdphi

def eps_SR(phi,mass2,vev):
    V = potential(phi,mass2,vev)
    dV = dVdphi(phi,mass2,vev)
    eps = 0.5*(dV**2/V**2)
    for i in range(0,len(eps)):
        if eps[i] < 10**(-30):
            eps[i] = 10**(-30)
            #print("FIXED THE ERROR ________________")
    return eps

def V_i_sum_sep(phi,mass2,vev,nfields):
    #V_vec = np.zeros(nfields)
    #for i in range(0,nfields):
    #    V_vec[i] = np.abs(mass2[i])/(4*vev[i]*vev[i])*(phi[i]**2-vev[i]**2)**2
    
    V_vec = np.abs(mass2)/(4*vev*vev)*(phi**2-vev**2)**2

    return(V_vec)

def Z_i_BE(phi_end,mass2,vev,nfields):
    #There was something in the fortran version about if HC_approx
    #Check this with Layne 
    #Z_vec = np.zeros(nfields)
    eps_end = eps_SR(phi_end,mass2,vev)
    eps = np.sum(eps_end)
    V = potential(phi_end,mass2,vev)
    V_vec = V_i_sum_sep(phi_end,mass2,vev,nfields)
    Z_vec = V*eps_end/eps - V_vec
    #for i in range(0,nfields):
    #    Z_vec[i] = V*eps_end[i]/eps - V_vec[i]
    return(Z_vec)    
    
def PR_SR(phi_pivot,phi_end,mass2,vev):
    V_piv = potential(phi_pivot,mass2,vev)
    H_piv = np.sqrt(V_piv/3)
    dN = dNdphi_SR(phi_pivot,phi_end,mass2,vev)
    P_dphi = (H_piv/2/math.pi)**2
    PR = np.sum(dN*dN)*P_dphi
    return(PR)


def d2Ndphi2_SR(phi_pivot,phi_end,mass2,vev,nfields):
    d2N = np.zeros((nfields,nfields))
    eta_piv = eta_SR(phi_pivot,mass2,vev,nfields)
    eps_piv = eps_SR(phi_pivot,mass2,vev)
    V_i_piv = V_i_sum_sep(phi_pivot,mass2,vev,nfields)
    Z_i = Z_i_BE(phi_end,mass2,vev,nfields)
    V_piv = potential(phi_pivot,mass2,vev)
    dZ_ij = dZdphi_ij_BE(phi_pivot,phi_end,mass2,vev,nfields)
    if V_piv < 10**(-30):
        V_piv = 10**(-30)
    for i in range(0,nfields):
        if eps_piv[i] < 10**(-30):
            eps_piv[i] = 10**(-30)
    delta = np.identity(nfields)
    for l in range(0,nfields):
        for k in range(0,nfields):
             d2N[l,k] = delta[l,k]*(1-(eta_piv[l]/2/eps_piv[l])*(V_i_piv[l]+Z_i[l])/V_piv) + (1/np.sqrt(2*eps_piv[l])/V_piv)*dZ_ij[l,k]
    return(d2N)

def dZdphi_ij_BE(phi_pivot,phi_end,mass2,vev,nfields):
    dZphi = np.zeros((nfields,nfields))
    eps_end = eps_SR(phi_end,mass2,vev)
    eps_piv = eps_SR(phi_pivot,mass2,vev)
    eps_t_end = np.sum(eps_end)
    V_end = potential(phi_end,mass2,vev)
    V_piv = potential(phi_pivot,mass2,vev)
    eta_end = eta_SR(phi_end,mass2,vev,nfields)
    for i in range(0,nfields):
        if eps_piv[i] < 10**(-30):
            eps_piv[i] = 10**(-30)
            #print("Did eps")
    if eps_t_end <10**(-30):
        eps_t_end = 10**(-30)
        #print("did epst")
    if V_piv < 10**(-30):
        V_piv = 10**(-30)
        #print("did vpiv")
        #CHECK THIS IS WHAT CALL CHECK ACTUALLY DOES
    delta = np.identity(nfields)
    for l in range(0,nfields):
        for k in range(0,nfields):
            for j in range(0,nfields):
                 dZphi[l,k] = dZphi[l,k] + (-V_end**2/V_piv)*np.sqrt(2/eps_piv[k])*(eps_end[j]*((eps_end[l]/eps_t_end) - delta[l,j])*((eps_end[k]/eps_t_end)-delta[k,j])*(1-(eta_end[j]/eps_t_end)))
    return(dZphi)

def fNL_SR(phi_pivot,phi_end,mass2,vev,nfields):
    dN = dNdphi_SR(phi_pivot,phi_end,mass2,vev)
    d2N = d2Ndphi2_SR(phi_pivot,phi_end,mass2,vev,nfields)
    fnl = 0.0
    for j in range(0,nfields):
        for k in range(0,nfields):
            fnl = fnl + dN[j]*dN[k]*d2N[j,k]
            #print(dN[j]*dN[k]*d2N[j,k])
    fnl = fnl*(-5/6)/(np.sum(dN*dN))**2
    return(fnl)
    
def r_SR(phi_pivot,phi_end,mass2,vev):
    dN = dNdphi_SR(phi_pivot,phi_end,mass2,vev)
    r = 8/np.sum(dN*dN)
    return(r)
    
def ns_SR(phi_pivot,phi_end,mass2,vev,nfields):
    dN = dNdphi_SR(phi_pivot,phi_end,mass2,vev)
    eps_piv = np.sum(eps_SR(phi_pivot,mass2,vev))
    V = potential(phi_pivot,mass2,vev)
    ns = 1 -2*eps_piv - 2/np.sum(dN*dN)
    d2V = d2Vdphi2(phi_pivot,mass2,vev,nfields)
    #BELOW WAS DONE AS IF d2V is a vector
    #NEEDS TO BE A MATRIX EQUATION IF V NOT SUM-SEPARABLE
    for i in range(0,nfields):
        ns = ns + 2*d2V[i]*dN[i]**2 /(V*np.sum(dN*dN))
    #print("ep,dn,V,d2V",eps_piv,dN,V,d2V)
    return(ns)
    

def xi_SR(phi,mass2,vev,nfields):
    d3V = d3Vdphi3(phi,mass2,vev,nfields)
    V = potential(phi,mass2,vev)
    dV = dVdphi(phi,mass2,vev)
    #xi=np.zeros(nfields)
    #BELOW WAS DONE AS IF d3V is a vector
    #NEEDS TO BE A MATRIX EQUATION IF V NOT SUM-SEPARABLE
    xi = dV*d3V**2/V**2
    #for i in range(0,nfields):
    #    xi[i]=dV[i]*d3V[i]**2/(V**2)
    return(xi)

def eta_SR(phi,mass2,vev,nfields):
    V = potential(phi,mass2,vev)
    d2V = d2Vdphi2(phi,mass2,vev,nfields)
    #eta=np.zeros(nfields)
    #BELOW WAS DONE AS IF d2V is a vector
    #NEEDS TO BE A MATRIX EQUATION IF V NOT SUM-SEPARABLE
    eta = d2V**2/V
    #for i in range(0,nfields):
    #    eta[i] = d2V[i]**2/V
    return(eta)

def alpha_s_SR(phi_pivot,phi_end,mass2,vev,nfields):
    V = potential(phi_pivot,mass2,vev)
    eps_i_piv = eps_SR(phi_pivot,mass2,vev)
    eps_piv = np.sum(eps_i_piv)
    xi_i_piv = xi_SR(phi_pivot,mass2,vev,nfields)
    eta_i_piv = eta_SR(phi_pivot,mass2,vev,nfields)
    V_i = V_i_sum_sep(phi_pivot,mass2,vev,nfields)
    Z_i = Z_i_BE(phi_pivot,mass2,vev,nfields)
    u_i = (V_i+Z_i)/V
    sum_ui_over_epsi = np.sum(u_i**2/eps_i_piv)
    #print(sum_ui_over_epsi)
    #print("epsipiv =",eps_i_piv)
    
    term1 = -8*eps_piv**2
    
    term2 = 4*np.sum(eps_i_piv*eta_i_piv)
    
    term3 = (-16/sum_ui_over_epsi**2)*(1-np.sum(eta_i_piv*u_i**2/(2*eps_i_piv)))**2

    term4 = (-8/sum_ui_over_epsi)*np.sum(eta_i_piv*u_i*(1-eta_i_piv*u_i**2/(2*eps_i_piv)))

    term5 = (4*eps_piv/sum_ui_over_epsi)*np.sum(eta_i_piv*u_i**2/eps_i_piv)
    
    term6 = (-2/sum_ui_over_epsi)*np.sum(xi_i_piv*u_i**2/eps_i_piv)
    #print(term1,term2,term3,term4,term5,term6)
    alpha_s = term1 + term2 + term3 + term4 + term5 + term6
    
    return(alpha_s)




In [ ]:
def potential(phi,mass2,vev):
    lmda4 = np.abs(mass2)/4.0/vev**2
    return np.sum(lmda4*(phi**2-vev**2)**2)

def dVdphi(phi,mass2,vev):
    lmda4 = np.abs(mass2)/4.0/vev**2
    return 4.0*lmda4*(phi**2-vev**2)*phi

def d2Vdphi2(phi,mass2,vev,nfields):
    d2V = np.zeros(nfields)
    #SUM-SEPARABLE SO WRITTEN AS VECTOR INSTEAD OF MATRIX
    #CHANGE THIS IF NOT SUMSEP!
    for i in range(0,nfields):
        d2V[i] = 4.0*np.abs(mass2[i])/(4*vev[i]*vev[i])*(3*phi[i]*phi[i]-vev[i]*vev[i])
    return(d2V)

def d3Vdphi3(phi,mass2,vev,nfields):
    d3V = np.zeros(nfields)
    #SUM-SEPARABLE SO WRITTEN AS VECTOR INSTEAD OF TENSOR
    #CHANGE THIS IF NOT SUMSEP!
    for i in range(0,nfields):
        d3V[i] = 4.0*np.abs(mass2[i])/(4*vev[i]*vev[i])*(6*phi[i])
    return(d3V)

def epsilon(dphidN):
    return 0.5*np.sum(np.array(dphidN)**2)

def slowrollEoM(phi,t0,mass2,vev):
    hubble2 = potential(phi,mass2,vev)/3.0
    
    return -dVdphi(phi,mass2,vev)/3.0/hubble2


In [ ]:
nfields=100
efolds_goal = 100
f = open('datanew.txt', 'w')

numb_samples = 100
for j in range(0,numb_samples):
    print(j)
    Npivot = np.random.uniform(40,70)
    g_beta = np.random.uniform(0.4,0.6)
    g_sigma = np.random.uniform(0.5,1)
    m_mu = np.random.uniform(-1,1)
    m_sigma = np.random.uniform(0.5,1)
    s = np.random.uniform(0.2,0.8)
    r = np.random.uniform(0.01,0.2)

    mass2 = GetMasses(nfields,g_beta,g_sigma,m_mu,m_sigma)
    vevs = np.random.uniform(s,s+r,nfields)
    (phi0,vevs) = GetPhi0s(nfields,efolds_goal,vevs,s,r)#also gets vevs
    vparams = (mass2,vevs)
    #print("Got all the sampling")

    tsteps = np.linspace(0,1.2*efolds_goal,efolds_goal*10)
    print("Antici-")
    traj = integrate.odeint(slowrollEoM,phi0,tsteps,args=vparams,tcrit=[0.0],rtol=1e-6,atol=1e-70)
    print("-pation")
    trajEps=[]
    for phi in traj:
        trajEps.append(epsilon(slowrollEoM(phi,0.0,mass2,vevs)))
    trajEps = np.array(trajEps)
    #plt.plot(trajEps)
    #print("got the trajectory")
    #traj = np.log10(traj)

    nSR = np.argmin(trajEps<1.0)
    if nSR==0: print("nsR = 0!!!")#nSR=len(trajEps)
    plotTraj = traj[0:nSR]
    #print(nSR)
    plotTime = tsteps[0:nSR]
    #print(plotTime)
    #print(plotTime[np.size(tsteps)-1])
    #Find Npivot as element of plotTime
    pivot_index = (np.abs(plotTime-(plotTime[nSR-1]-Npivot))).argmin()

    nTotal = plotTime[-1]
    #print("Number of efolds =",nTotal)

    end_index = (np.abs(plotTime-(plotTime[nSR-1]))).argmin()
    #print(pivot_index)
    #print(end_index)   
    phi_pivot = np.zeros(nfields)
    phi_end = np.zeros(nfields)

    for i in range(0,nfields):
        phi_pivot[i] = plotTraj[pivot_index,i]
        phi_end[i] = plotTraj[end_index,i]
    #print(phi_pivot)

    wigrad = np.amax(abs(mass2))
    ns = ns_SR(phi_pivot,phi_end,mass2,vevs,nfields)
    #print("ns =",ns)
    alpha = alpha_s_SR(phi_pivot,phi_end,mass2,vevs,nfields)
    r_val = r_SR(phi_pivot,phi_end,mass2,vevs)
    print("stalling")
    fnl = fNL_SR(phi_pivot,phi_end,mass2,vevs,nfields)
    print("here")
    pr = PR_SR(phi_pivot,phi_end,mass2,vevs)
    #zij = dZdphi_ij_BE(phi_pivot,phi_end,mass2,vevs,nfields)
    #print("zij=",zij)
    #print("r =",r_val)
    #print("fnl =",fnl)
    #print("pr =",pr)
    f.write(str(Npivot) + " " + str(g_beta) + " " + str(ns) + " " + str(alpha) + " " + str(r_val) + " " + str(fnl) + " " + str(pr) + " " + str(wigrad) + "\n")
    #print(str(Npivot) + " " + str(g_beta) + " " + str(ns) + " " + str(alpha) + " " + str(r_val) + " " + str(fnl) + " " + str(pr) + " " + str(wigrad) + "\n")
#fnl, r, Ps, radius wigner, gsigma, massmu, masssigma 



f.close()
#print("alpha_s =",alpha)
#testPlot = np.random.choice(range(0,nfields),5)

#print(testPlot)

#for plotter in testPlot:
#    plt.plot(plotTime,(plotTraj[:,plotter]))
#    plt.plot(plotTime,(plotTraj[:,plotter]))
#    #print(plotTraj[pivot_index,plotter])


#plt.plot(plotTime,trajEps[0:nSR])

0
Antici-


/Users/chriskervick/anaconda/lib/python3.5/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


-pation
stalling
here
1
Antici-
-pation
stalling
here
2
Antici-
-pation
stalling
here
3
Antici-
-pation
stalling
here
4
Antici-
-pation
stalling
here
5
Antici-
-pation
stalling
here
6
Antici-
-pation
stalling
here
7
Antici-
-pation
stalling
here
8
Antici-
-pation
stalling
here
9
Antici-
-pation
stalling
here
10
Antici-
-pation
stalling
here
11
Antici-
-pation
stalling
here
12
Antici-
-pation
stalling
here
13
Antici-
-pation
stalling
here
14
Antici-
-pation
stalling
here
15
Antici-
-pation
stalling
here
16
Antici-
-pation
stalling
here
17
Antici-
-pation
stalling
here
18
Antici-
-pation
stalling
here
19
Antici-
-pation
stalling
here
20
Antici-
-pation
stalling
here
21
Antici-
